# Scene self-test

Cell 1 prints a compact scene summary (paths, counts, bounds, positions).
Cell 2 renders a 3D overlay (buildings + BS + UAV init + UAV altitude band).


In [1]:
import json
from pathlib import Path
import numpy as np
import plotly.io as pio

from notebook_utils import load_buildings_json, print_topk_tallest_buildings, visualize_scene_selftest_from_cfg

BASE = Path("~/Projects/metaRL_merged").expanduser()

# ---- select one config ----
# CFG_PATH = BASE / "inner_loop/model_a/configs/raleigh_modelA.json"
# CFG_PATH = BASE / "inner_loop/model_a/configs/atlanta_modelA.json"
# CFG_PATH = BASE / "inner_loop/model_a/configs/boston_modelA.json"
CFG_PATH = BASE / "inner_loop/model_a/configs/chapel_modelA.json"

with open(CFG_PATH, "r", encoding="utf-8") as f:
    cfg = json.load(f)

# ---- resolve key paths ----
PROJECT_ROOT = Path(cfg.get("project_root", str(BASE))).expanduser()
SCENE_XML = (PROJECT_ROOT / cfg["scene_xml"]).resolve()

BUILDINGS_JSON = (PROJECT_ROOT / cfg["buildings"]["boxes_json"]).resolve()

# ---- parse key blocks ----
uav_cfg = cfg["uav"]
bs_cfg = cfg["bs"]
gu_cfg = cfg["gu"]

n_uav = int(uav_cfg.get("n_uav", len(uav_cfg.get("init_xyz", []))))
uav_init_xyz = uav_cfg.get("init_xyz", [])
uav_bounds = uav_cfg.get("bounds", None)
uav_margin = uav_cfg.get("building_margin", None)

bs_positions = bs_cfg.get("positions", [])
n_bs = len(bs_positions)

n_gus = int(gu_cfg.get("num_gus", 0))
gu_xy_region = gu_cfg.get("xy_region", None)
gu_height = gu_cfg.get("height", None)

# ---- load buildings json to get scene extents ----
bd = load_buildings_json(str(BUILDINGS_JSON))
bs = bd.buildings

xs, ys, zs = [], [], []
for b in bs:
    try:
        xs += [float(b["xmin"]), float(b["xmax"])]
        ys += [float(b["ymin"]), float(b["ymax"])]
        zmin = float(b.get("zmin", 0.0))
        zmax = float(b.get("zmax", b.get("roof_zmax", zmin)))
        zs += [zmin, zmax]
    except Exception:
        pass

x_rng = (min(xs), max(xs)) if xs else (None, None)
y_rng = (min(ys), max(ys)) if ys else (None, None)
z_rng = (min(zs), max(zs)) if zs else (None, None)

scene_size = None
if xs and ys and zs:
    scene_size = (x_rng[1] - x_rng[0], y_rng[1] - y_rng[0], z_rng[1] - z_rng[0])

# ---- print compact report ----
print("=== Scene self-test summary ===")
print("config_path:", CFG_PATH)
print("project_root:", PROJECT_ROOT)
print("scene_xml:", SCENE_XML)
print("buildings_json:", BUILDINGS_JSON)
print()

print("--- counts ---")
print("num_buildings:", len(bs))
print("num_uav:", n_uav)
print("num_gus:", n_gus)
print("num_bs:", n_bs)
print()

print("--- extents (from buildings json) ---")
print("x_range:", x_rng)
print("y_range:", y_rng)
print("z_range:", z_rng)
print("scene_size_xyz:", scene_size)
print()

print("--- UAV config ---")
# UAV height range (from bounds z-range)
uav_height_range = None
if uav_bounds is not None and len(uav_bounds) == 3 and len(uav_bounds[2]) == 2:
    uav_height_range = (float(uav_bounds[2][0]), float(uav_bounds[2][1]))

print("uav_bounds:", uav_bounds)
print("uav_height_range:", uav_height_range)
# print("uav_init_xyz (first 5):", uav_init_xyz[:5])
print("building_margin:", uav_margin)
print()

print("--- BS config ---")
print("bs_positions:", bs_positions)
print("aim_mode:", bs_cfg.get("aim_mode", None))
print()

print("--- GU config ---")
print("gu_xy_region:", gu_xy_region)
print("gu_height:", gu_height)
print()

=== Scene self-test summary ===
config_path: /home/jhuang52/Projects/metaRL_merged/inner_loop/model_a/configs/chapel_modelA.json
project_root: /home/jhuang52/Projects/metaRL_merged
scene_xml: /data/jhuang52/Projects/metaRL_merged/scenarios/Chapel-Hill-(Campus)/final-scene.xml
buildings_json: /data/jhuang52/Projects/metaRL_merged/scenarios/chapel_buildings.json

--- counts ---
num_buildings: 1361
num_uav: 5
num_gus: 150
num_bs: 2

--- extents (from buildings json) ---
x_range: (-996.4613647460938, 988.262451171875)
y_range: (-919.8028564453125, 931.1636352539062)
z_range: (0.0, 102.90943908691406)
scene_size_xyz: (1984.7238159179688, 1850.9664916992188, 102.90943908691406)

--- UAV config ---
uav_bounds: [[-1000, 1000], [-1000, 1000], [50, 200]]
uav_height_range: (50.0, 200.0)
building_margin: 1.0

--- BS config ---
bs_positions: [[60, 680, 95], [-760, -210, 110]]
aim_mode: none

--- GU config ---
gu_xy_region: [[-800, 800], [-800, 800]]
gu_height: 1.5



In [2]:
top5_building_ids, top5_heights = print_topk_tallest_buildings(BUILDINGS_JSON, k=5)

--- Top-K tallest buildings (by height=zmax-zmin) ---
[1] id=element_1263
     height=102.91, zmin=0.00, zmax=102.91
     aabb=([-777.2,-751.6], [-229.3,-203.7])
     center=[-764.4140930175781, -216.51783752441406, 51.45471954345703]
[2] id=element_1315
     height=86.39, zmin=0.00, zmax=86.39
     aabb=([-548.8,-456.5], [309.7,390.5])
     center=[-502.65309143066406, 350.1180877685547, 43.196895599365234]
[3] id=element_999
     height=84.78, zmin=0.00, zmax=84.78
     aabb=([-46.7,16.9], [655.0,694.6])
     center=[-14.899064064025879, 674.7796325683594, 42.390342712402344]
[4] id=element_1321
     height=82.38, zmin=0.00, zmax=82.38
     aabb=([29.5,97.5], [665.3,704.7])
     center=[63.50974941253662, 685.0384826660156, 41.18825149536133]
[5] id=element_1320
     height=81.28, zmin=0.00, zmax=81.28
     aabb=([53.0,90.4], [570.9,631.1])
     center=[71.67941856384277, 600.9942932128906, 40.63993835449219]


In [3]:
fig = visualize_scene_selftest_from_cfg(CFG_PATH, highlight_building_ids=top5_building_ids)
fig.show(renderer="iframe")